In [10]:
import torch.nn as nn

class LinearWithConstraint(nn.Linear):
    def __init__(self, *args, doWeightNorm=True, max_norm=1., **kwargs):
        self.max_norm = max_norm
        self.doWeightNorm = doWeightNorm
        super(LinearWithConstraint, self).__init__(*args, **kwargs)

    def forward(self, x):
        if self.doWeightNorm:
            self.weight.data = torch.renorm(
                self.weight.data, p=2, dim=0, maxnorm=self.max_norm
            )
        return super(LinearWithConstraint, self).forward(x)
linear = LinearWithConstraint(5, 3, doWeightNorm=True, max_norm=0.5)

x = torch.randn(2, 5)
output = linear(x)
print(output)  # 输出线性层的结果

tensor([[-1.4581, -0.5450, -0.1571],
        [-0.1612, -0.2246,  0.2249]], grad_fn=<AddmmBackward0>)


## 模型理解

### 指标计算

In [3]:
# dataset bci42a 9 subject
data = [0.776,0.5469,0.8125,0.5851,0.6285,0.651,0.7569,0.7674,0.7309]
avg = sum(data)/len(data)
std = (sum((x - avg) ** 2 for x in data) / len(data)) ** 0.5
print(f"Average: {avg:.4f}, Standard Deviation: {std:.4f}")

Average: 0.6950, Standard Deviation: 0.0889


In [ ]:
# p-value calculation
from scipy.stats import wilcoxon
model_A = [85.0, 72.0, 90.5, 65.0, 78.0]
model_B = [88.0, 73.5, 89.0, 68.0, 75.1]

# 步骤 2-5
# 执行 Wilcoxon 符号秩检验
# result.pvalue 就是我们要找的 P 值
statistic, p_value = wilcoxon(model_B, model_A)
# 输出
print(f"Wilcoxon W 统计量: {statistic}")  # W 统计量: 2.5
print(f"P 值: {p_value}")                   # P 值: 0.125
# 最终判断: 
if p_value < 0.05:
    print("差异具有统计显著性 (P < 0.05)")
else:
    print("差异不具有统计显著性 (P > 0.05)")


Wilcoxon W 统计量: 4.5
P 值: 0.5625
差异不具有统计显著性 (P > 0.05)


### 权重加载

In [8]:
path = r'output/bci42a/2025-11-19--10-33-46B7/sub0\checkpoint.pth.tar'
import torch
checkpoint = torch.load(path, map_location='cpu')
print(checkpoint.keys())
# for key,_ in checkpoint['state_dict'].items():
#     print(key)

dict_keys(['state_dict'])


### domain shift


度量不同数据集之间的域偏移程度分为：数据级，特征级和性能级

1. 数据级：

- MMD： 测量两个分布在RKHS（再生希伯特空间）中的距离

- EM 距离


图：
原本不同的数据集之间的域偏移程度<---->不同数据集之间的域偏移程度

#### 域偏移衡量算法

### 改进COCO


对比学习，概念级别对齐

Average: 0.6950222222222222, Standard Deviation: 0.08888150385988675


### EEG-DG



**单受试者场景下**训练域泛化模型:

通过利用 EEG 随时间变化的特性，人为将一次实验数据拆分成两个“风格”略有不同的域，强迫模型学习抵抗时间漂移的鲁棒特征

    - 前半段（Domain 1）： 受试者精力充沛，电极接触阻抗可能较低，注意力集中。

    - 后半段（Domain 2）： 受试者可能出现疲劳、注意力分散、或者头皮出汗导致电极阻抗变化。

随时间产生的信号漂移

数据加载测试

In [ ]:
path = '/home/yaoyuan/EEG/Datasets/bci42a/rawMat/s001.mat'

import scipy.io

data = scipy.io.loadmat(path)  # 读取.mat文件

print(data.keys())      # 输出所有的键
feature= data['x']      # 假设数据存储在键 'data' 下
print(feature.shape)    # 输出数据的形状
label = data['y']
print(label)            # 输出标签的形状

dict_keys(['__header__', '__version__', '__globals__', 'x', 'y', 'c', 's'])
(22, 1000, 288)
[[3 2 1 0 0 1 2 3 1 2 0 0 0 3 1 1 0 0 2 0 1 3 3 2 0 3 3 1 3 3 1 0 1 2 2 2
  3 2 0 3 1 2 1 2 3 1 2 0 0 0 3 1 0 2 0 2 1 3 0 2 2 0 2 1 3 3 3 2 0 3 1 3
  1 0 2 1 0 2 2 0 2 3 3 1 0 1 3 1 3 2 1 1 1 2 3 0 1 3 0 2 2 3 0 0 2 1 3 3
  3 1 0 2 1 3 0 3 2 1 3 3 0 1 1 2 3 1 0 0 3 1 0 2 1 1 2 0 3 2 2 2 2 0 1 0
  1 0 0 2 2 1 2 3 0 3 0 0 1 3 2 1 3 2 3 2 3 1 1 3 0 1 1 1 2 3 0 3 0 2 0 3
  0 2 0 1 2 2 3 0 1 3 1 2 2 0 3 1 3 0 0 2 2 1 3 1 1 0 1 3 3 1 1 1 1 3 3 2
  3 0 1 2 1 0 3 0 3 0 0 0 0 2 2 3 1 2 2 2 3 2 0 2 0 3 1 3 3 2 3 3 2 1 3 2
  0 1 1 1 2 1 3 2 3 1 2 0 3 0 2 3 0 2 0 1 1 0 3 0 3 2 2 0 2 1 1 0 2 0 1 0]]


In [5]:
import scipy.io as scio

path = '/home/yaoyuan/EEG/Datasets/bci42a/rawMat/'


data_train = scio.loadmat(f'{path}/s001.mat') # 假设有 s001
data_eval = scio.loadmat(f'{path}/se001.mat')

# 打印查看 Key 或 形状
print("Train Keys:", data_train.keys())
print("Eval Keys:", data_eval.keys())

Train Keys: dict_keys(['__header__', '__version__', '__globals__', 'x', 'y', 'c', 's'])
Eval Keys: dict_keys(['__header__', '__version__', '__globals__', 'x', 'y', 'c', 's'])


In [9]:
import torch

a = torch.randn((5, 3))
b = a.unsqueeze(1)
print(a.unsqueeze(1).expand(-1,a.shape[1],-1).shape)  # 输出: torch.Size([5, 3])
print(a.shape, b.shape)  # 输出: torch.Size([5, 3]) torch.Size([5, 1, 3])

torch.Size([5, 3, 3])
torch.Size([5, 3]) torch.Size([5, 1, 3])
